In [1]:
#patching segment_anything.mask_decoder to support 5‑D src tensors
import glob, re, os

cands = glob.glob("/usr/local/lib/python3.*/dist-packages/segment_anything/modeling/mask_decoder.py")
if not cands:
    raise FileNotFoundError("Can't find segment_anything/modeling/mask_decoder.py")
decoder = cands[0]
print("Patching:", decoder)

#read & patch
with open(decoder, "r") as f:
    lines = f.readlines()

patched = []
for line in lines:
    #replace 4‑D unpack with 5‑D + collapse
    if re.match(r"\s*b, c, h, w = src\.shape", line):
        indent = line[:line.index("b")]
        patched += [
            f"{indent}# accommodate extra token dim\n",
            f"{indent}b, t, c, h, w = src.shape\n",
            f"{indent}src = src.view(b * t, c, h, w)\n",
            f"{indent}pos_src = pos_src.view(b * t, c, h, w)\n",
        ]
    else:
        patched.append(line)

final = []
for line in patched:
    if line.strip() == "return masks, iou_pred":
        indent = line[:line.index("return")]
        final += [
            f"{indent}# reshape masks and iou back to [B, T, …]\n",
            f"{indent}masks = masks.view(b, t, 1, h, w)\n",
            f"{indent}iou_pred = iou_pred.view(b, t)\n",
        ]
        final.append(line)
    else:
        final.append(line)

with open(decoder, "w") as f:
    f.writelines(final)

print("Patched segment_anything.mask_decoder – now restart the kernel.")


Patching: /usr/local/lib/python3.11/dist-packages/segment_anything/modeling/mask_decoder.py
Patched segment_anything.mask_decoder – now restart the kernel.


In [14]:
PROJECT_PATH = "/kaggle/input/task-aware-sam-lora/task_aware_sam_lora"
import sys, os
sys.path.append(PROJECT_PATH)

!pip install -q transformers accelerate opencv-python matplotlib wandb datasets fsspec==2023.9.2 pycocotools

SAM_CKPT_PATH = "/kaggle/working/sam_vit_h_4b8939.pth"
if not os.path.exists(SAM_CKPT_PATH):
    !wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P /kaggle/working/


In [15]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


Using device: cuda
GPU: Tesla T4
GPU Memory: 14.7 GB


In [16]:
from src.models.hypernetwork import TaskAwareHyperNet, create_sam_lora_config
from src.models.lora_adapter import LoRAAdapter, LoRAConfig
from src.models.sam_wrapper import SAMWithLoRA, TaskAwareSAM
from src.data.dataset import TaskAwareDataset
from src.utils.text_processing import TaskDescriptionProcessor
from config.train_config import get_t4_optimized_config


In [4]:
#config initialization
#load config
base_cfg   = get_t4_optimized_config()
model_cfg  = base_cfg.model
system_cfg = base_cfg.system

#LoRA adapter config
lora_cfg = LoRAConfig(
    rank=model_cfg.lora_rank,
    alpha=model_cfg.lora_alpha,
    dropout=model_cfg.lora_dropout,
    target_modules=model_cfg.lora_target_modules
)

#SAM + LoRA
sam_model = SAMWithLoRA(
    sam_checkpoint=SAM_CKPT_PATH,
    model_type=model_cfg.sam_model_type, 
    lora_config=lora_cfg,
    device=device
)

#hypernetwork
lora_config_dict = create_sam_lora_config(sam_model.sam.mask_decoder)
hypernet = TaskAwareHyperNet(
    lora_config=lora_config_dict,
    lora_rank=model_cfg.lora_rank,
    text_encoder_model=model_cfg.text_encoder_model,
    hidden_dim=model_cfg.hypernetwork_hidden_dim,
    num_layers=model_cfg.hypernetwork_num_layers,
    num_heads=model_cfg.hypernetwork_num_heads,
    dropout=model_cfg.hypernetwork_dropout
).to(device)

text_processor = TaskDescriptionProcessor()


Frozen SAM encoders
Added LoRA to: output_hypernetworks_mlps.0.layers.0 (as output_hypernetworks_mlps_0_layers_0)
Added LoRA to: output_hypernetworks_mlps.0.layers.1 (as output_hypernetworks_mlps_0_layers_1)
Added LoRA to: output_hypernetworks_mlps.0.layers.2 (as output_hypernetworks_mlps_0_layers_2)
Added LoRA to: output_hypernetworks_mlps.1.layers.0 (as output_hypernetworks_mlps_1_layers_0)
Added LoRA to: output_hypernetworks_mlps.1.layers.1 (as output_hypernetworks_mlps_1_layers_1)
Added LoRA to: output_hypernetworks_mlps.1.layers.2 (as output_hypernetworks_mlps_1_layers_2)
Added LoRA to: output_hypernetworks_mlps.2.layers.0 (as output_hypernetworks_mlps_2_layers_0)
Added LoRA to: output_hypernetworks_mlps.2.layers.1 (as output_hypernetworks_mlps_2_layers_1)
Added LoRA to: output_hypernetworks_mlps.2.layers.2 (as output_hypernetworks_mlps_2_layers_2)
Added LoRA to: output_hypernetworks_mlps.3.layers.0 (as output_hypernetworks_mlps_3_layers_0)
Added LoRA to: output_hypernetworks_mlps

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

2025-07-19 04:34:04.626799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752899644.848224      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752899644.909247      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

TaskAwareHyperNet initialized with 26128 LoRA parameters
  output_hypernetworks_mlps.0.layers.0: 256x256, LoRA A: 1024, LoRA B: 1024
  output_hypernetworks_mlps.0.layers.1: 256x256, LoRA A: 1024, LoRA B: 1024
  output_hypernetworks_mlps.0.layers.2: 256x32, LoRA A: 1024, LoRA B: 128
  output_hypernetworks_mlps.1.layers.0: 256x256, LoRA A: 1024, LoRA B: 1024
  output_hypernetworks_mlps.1.layers.1: 256x256, LoRA A: 1024, LoRA B: 1024
  output_hypernetworks_mlps.1.layers.2: 256x32, LoRA A: 1024, LoRA B: 128
  output_hypernetworks_mlps.2.layers.0: 256x256, LoRA A: 1024, LoRA B: 1024
  output_hypernetworks_mlps.2.layers.1: 256x256, LoRA A: 1024, LoRA B: 1024
  output_hypernetworks_mlps.2.layers.2: 256x32, LoRA A: 1024, LoRA B: 128
  output_hypernetworks_mlps.3.layers.0: 256x256, LoRA A: 1024, LoRA B: 1024
  output_hypernetworks_mlps.3.layers.1: 256x256, LoRA A: 1024, LoRA B: 1024
  output_hypernetworks_mlps.3.layers.2: 256x32, LoRA A: 1024, LoRA B: 128
  iou_prediction_head.layers.0: 256x256

In [5]:
#transforms/loader
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor(),
])

COCO_DIR = "/kaggle/input/coco-2017-dataset/coco2017"
coco_dataset = TaskAwareDataset(
    data_dir=COCO_DIR,
    split="train",
    mode="instance",
    transform=transform,
    max_samples=500
)

from torch.utils.data import DataLoader
train_dataloader = DataLoader(
    coco_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)


loading annotations into memory...
Done (t=21.56s)
creating index...
index created!


In [6]:
# train fcn
import os
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm

def to_str_list(x):
    print(f"[DEBUG] raw task_description type: {type(x)}, value: {x}")
    if isinstance(x, str):
        return [x]
    if isinstance(x, torch.Tensor):
        if x.dtype == torch.object:
            return [str(xx) for xx in x]
        if x.numel() == 1:
            return [str(x.item())]
        else:
            return [str(xx.item()) for xx in x]
    if isinstance(x, bytes):
        return [x.decode("utf8")]
    if isinstance(x, tuple):
        return [str(xx) for xx in x]
    if isinstance(x, list):
        if all(isinstance(xx, list) for xx in x):
            return [str(jj) for xx in x for jj in xx]
        if all(isinstance(xx, torch.Tensor) for xx in x):
            return [str(xx.item()) if xx.numel() == 1 else str(xx) for xx in x]
        if all(isinstance(xx, bytes) for xx in x):
            return [xx.decode("utf8") for xx in x]
        if all(isinstance(xx, str) for xx in x):
            return x
        return [str(xx) for xx in x]
    return [str(x)]

optimizer = torch.optim.AdamW(
    hypernet.get_hypernetwork_params(),
    lr=base_cfg.training.learning_rate,
    weight_decay=base_cfg.training.weight_decay
)

def train_epoch(hypernetwork, sam_model, dataloader, optimizer, epoch, device):
    hypernetwork.train()
    sam_model.train()
    total_loss = 0

    pbar = tqdm(dataloader, desc=f'Epoch {epoch+1}')
    for batch_idx, batch in enumerate(pbar):
        images = batch['image'].to(device)          
        masks = batch['mask'].to(device)             
        raw_task_desc = batch['task_description']

        task_descriptions = to_str_list(raw_task_desc)
        print(f"[DEBUG] converted task_descriptions type: {type(task_descriptions)} value: {task_descriptions}")

        optimizer.zero_grad()

        # get LoRA weights from hypernetwork
        lora_weights = hypernetwork(task_descriptions)

        # debug: print out each adapter’s provided shape
        total_provided = 0
        print("LoRA weights from hypernetwork:")
        for name, w in lora_weights.items():
            print(f"  • {name:40s} provided → {tuple(w.shape)}  (numel={w.numel()})")
            total_provided += w.numel()
        print(f"  → total provided params: {total_provided}\n")

        #apply LoRA weights
        sam_model.apply_lora(lora_weights)

        #get and collect embeddings, squeezing out the extra dim
        image_embeddings = []
        for i in range(images.size(0)):
            img_np = images[i].permute(1, 2, 0).cpu().numpy()
            img_np = (img_np * 255).astype(np.uint8)
            emb = sam_model.get_image_embeddings(img_np)    
            emb = emb.squeeze(0)                            
            image_embeddings.append(emb)
        image_embeddings = torch.stack(image_embeddings).to(device)  

        #prepare points
        points = batch.get('points', None)
        point_labels = batch.get('point_labels', None)
        if points is None or point_labels is None:
            b, c, h, w = images.shape
            points = torch.tensor([[[w//2, h//2]]]*b).float().to(device)
            point_labels = torch.ones((b,1), dtype=torch.long).to(device)
        else:
            points = points.to(device)
            point_labels = point_labels.to(device)

        #forward + loss
        with torch.cuda.amp.autocast():
            predicted_masks, _ = sam_model(
                image_embeddings=image_embeddings,
                point_coords=points,
                point_labels=point_labels,
                multimask_output=False
            )  #predicted_masks: [B,1,256,256]

            #resize ground-truth masks to match output
            gt_masks = masks.unsqueeze(1).float()  
            gt_masks = F.interpolate(
                gt_masks,
                size=predicted_masks.shape[-2:],   #(256,256)
                mode='nearest'
            ) 

            loss = F.binary_cross_entropy_with_logits(predicted_masks, gt_masks)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix({'loss': loss.item()})

        if batch_idx % 10 == 0:
            torch.cuda.empty_cache()

    return total_loss / len(dataloader)


In [8]:
# train
import os
import torch
dataloader = train_dataloader

ckpt_dir = "/mnt/data/checkpoints"
os.makedirs(ckpt_dir, exist_ok=True)

def train_model(epochs, dataloader):
    print("Starting training…")
    for epoch in range(epochs):
        avg_loss = train_epoch(hypernet, sam_model, dataloader, optimizer, epoch, device)
        print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")
        # save every epoch
        checkpoint = {
            'epoch': epoch,
            'hypernetwork_state_dict': hypernet.state_dict(),
            'sam_state_dict': sam_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss
        }
        path = f"{ckpt_dir}/checkpoint_epoch_{epoch}.pth"
        torch.save(checkpoint, path)
        print(f" ✔️Saved checkpoint to {path}")
    print("Training completed!")

train_model(epochs=2, dataloader=train_dataloader)

Starting training…


Epoch 1:   0%|          | 0/500 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[DEBUG] raw task_description type: <class 'list'>, value: ['segment all baseball bat']
[DEBUG] converted task_descriptions type: <class 'list'> value: ['segment all baseball bat']
LoRA weights from hypernetwork:
  • output_hypernetworks_mlps.0.layers.0.lora_A provided → (1, 256, 4)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.0.lora_B provided → (1, 4, 256)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.1.lora_A provided → (1, 256, 4)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.1.lora_B provided → (1, 4, 256)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.2.lora_A provided → (1, 256, 4)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.2.lora_B provided → (1, 4, 32)  (numel=128)
  • output_hypernetworks_mlps.1.layers.0.lora_A provided → (1, 256, 4)  (numel=1024)
  • output_hypernetworks_mlps.1.layers.0.lora_B provided → (1, 4, 256)  (numel=1024)
  • output_hypernetworks_mlps.1.layers.1.lora_A provided → (1, 256, 4)  (numel=1024)
  • output_hypernetworks_

Epoch 1:   0%|          | 1/500 [00:02<22:42,  2.73s/it, loss=0.15]

[DEBUG] image_embeddings shape: torch.Size([1, 256, 64, 64])
[DEBUG] point_coords shape: torch.Size([1, 5, 2])
[DEBUG] point_labels shape: torch.Size([1, 5])
[DEBUG] sparse_embeddings shape: torch.Size([1, 6, 256])
[DEBUG] dense_embeddings shape: torch.Size([1, 256, 64, 64])
[DEBUG] image_pe shape: torch.Size([1, 256, 64, 64])
[DEBUG] dense_embeddings final shape: torch.Size([1, 256, 64, 64])
[DEBUG] raw task_description type: <class 'list'>, value: ['segment all frisbee']
[DEBUG] converted task_descriptions type: <class 'list'> value: ['segment all frisbee']
LoRA weights from hypernetwork:
  • output_hypernetworks_mlps.0.layers.0.lora_A provided → (1, 256, 4)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.0.lora_B provided → (1, 4, 256)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.1.lora_A provided → (1, 256, 4)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.1.lora_B provided → (1, 4, 256)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.2.lora_A provided → (1,

...

Epoch 2: 100%|█████████▉| 499/500 [18:56<00:02,  2.28s/it, loss=0.709]

[DEBUG] image_embeddings shape: torch.Size([1, 256, 64, 64])
[DEBUG] point_coords shape: torch.Size([1, 5, 2])
[DEBUG] point_labels shape: torch.Size([1, 5])
[DEBUG] sparse_embeddings shape: torch.Size([1, 6, 256])
[DEBUG] dense_embeddings shape: torch.Size([1, 256, 64, 64])
[DEBUG] image_pe shape: torch.Size([1, 256, 64, 64])
[DEBUG] dense_embeddings final shape: torch.Size([1, 256, 64, 64])
[DEBUG] raw task_description type: <class 'list'>, value: ['locate all person objects']
[DEBUG] converted task_descriptions type: <class 'list'> value: ['locate all person objects']
LoRA weights from hypernetwork:
  • output_hypernetworks_mlps.0.layers.0.lora_A provided → (1, 256, 4)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.0.lora_B provided → (1, 4, 256)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.1.lora_A provided → (1, 256, 4)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.1.lora_B provided → (1, 4, 256)  (numel=1024)
  • output_hypernetworks_mlps.0.layers.2.lora_A pr

Epoch 2: 100%|██████████| 500/500 [18:59<00:00,  2.28s/it, loss=1.48] 

[DEBUG] image_embeddings shape: torch.Size([1, 256, 64, 64])
[DEBUG] point_coords shape: torch.Size([1, 5, 2])
[DEBUG] point_labels shape: torch.Size([1, 5])
[DEBUG] sparse_embeddings shape: torch.Size([1, 6, 256])
[DEBUG] dense_embeddings shape: torch.Size([1, 256, 64, 64])
[DEBUG] image_pe shape: torch.Size([1, 256, 64, 64])
[DEBUG] dense_embeddings final shape: torch.Size([1, 256, 64, 64])
Epoch 2/2, Average Loss: 1.7307


 ✔️Saved checkpoint to /mnt/data/checkpoints/checkpoint_epoch_1.pth
Training completed!


In [25]:
from IPython.display import HTML, FileLink

display(HTML('<a href="checkpoints.zip" download>⬇️ Download checkpoints.zip</a>'))

#link to the SAM backbone checkpoint
display(HTML('<a href="sam_vit_h_4b8939.pth" download>⬇️ Download sam_vit_h_4b8939.pth</a>'))

#alternative
print("Or use FileLink widgets below:")
display(FileLink('checkpoints.zip'))
display(FileLink('sam_vit_h_4b8939.pth'))



Or use FileLink widgets below:


/kaggle/working/checkpoints.zip

/kaggle/working/sam_vit_h_4b8939.pth